In [1]:
pip install pyspark

Note: you may need to restart the kernel to use updated packages.


In [2]:
# Librerie
from pyspark.sql import SparkSession
import pyspark.sql.types as T
import pyspark.sql.functions as F

In [3]:
# Creazione SparkSession
spark = (SparkSession.builder.appName("Guitars").getOrCreate())

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


23/03/05 15:29:15 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [4]:
# Path dei csv files di nostro interesse
path_prodotti = "/kaggle/input/chitarre/Prodotti.csv"
path_vendite = "/kaggle/input/chitarre/Vendite.csv"
path_venditori = "/kaggle/input/chitarre/Venditori.csv"

In [5]:
# Creazione dei df

prodotti_df = (spark.read.format("csv")
              .option("header", True)
              .option("delimiter", ";")
              .load(path_prodotti))

vendite_df = (spark.read.format("csv")
              .option("header", True)
              .option("delimiter", ";")
              .load(path_vendite))

venditori_df = (spark.read.format("csv")
              .option("header", True)
              .option("delimiter", ";")
              .load(path_venditori))

prodotti_df.show(5), vendite_df.show(5), venditori_df.show(5)

+---+------+------------+------+---------+
| ID| Marca|     Modello|Prezzo|Categoria|
+---+------+------------+------+---------+
|  1|Gibson|    Les Paul|  2300| Chitarre|
|  2|Gibson|          SG|  1700| Chitarre|
|  3|Gibson|        J-45|  1900| Chitarre|
|  4|Fender|  Telecaster|  1800| Chitarre|
|  5|Fender|Stratocaster|  1900| Chitarre|
+---+------+------------+------+---------+
only showing top 5 rows

+----------+---------+--------+--------+
|      Data|Venditore|Prodotto|Quantita|
+----------+---------+--------+--------+
|11/01/2022|        2|       1|       1|
|21/02/2022|        3|       6|       2|
|24/03/2022|        1|      11|       2|
|02/04/2022|        4|       2|       4|
|12/04/2022|        8|       3|       2|
+----------+---------+--------+--------+
only showing top 5 rows

+---+------+--------+
| ID|  Nome| Cognome|
+---+------+--------+
|  1|Andrea|  Assini|
|  2| Bruno| Bettini|
|  3| Carlo|Cicerini|
|  4| Dario|  Desini|
|  5| Elena| Ettorre|
+---+------+------

(None, None, None)

In [6]:
# Contare quante righe contiene ogni tabella
print("Prodotti: " + str(prodotti_df.count()))
print("Vendite: " + str(vendite_df.count()))
print("Venditori: " + str(venditori_df.count()))

Prodotti: 12
Vendite: 30
Venditori: 8


In [7]:
# Numero di vendite, anno per anno

vendite_df = vendite_df\
                .withColumn("Data", F.to_date(F.col("Data"), "dd/MM/yyyy"))\
                .withColumn("Anno", F.year(F.col("Data")))

vendite_per_anno_df = vendite_df\
                .groupBy(F.col("Anno"))\
                .agg(F.count("*").alias("Vendite nell'anno"))\
                .orderBy("Anno")

vendite_per_anno_df.show()

+----+-----------------+
|Anno|Vendite nell'anno|
+----+-----------------+
|2022|               15|
|2023|               15|
+----+-----------------+



In [8]:
# Numero di vendite, per marca

vendite_per_marca_df = vendite_df\
                    .join(prodotti_df, vendite_df["Prodotto"] == prodotti_df["ID"])\
                    .groupBy(F.col("Marca"))\
                    .agg(F.count("*"))\
                    .orderBy("Marca")

vendite_per_marca_df.show()

+--------+--------+
|   Marca|count(1)|
+--------+--------+
|  Fender|      10|
|  Gibson|       9|
|  Ibanez|       8|
|MusicMan|       3|
+--------+--------+



In [9]:
# Numero di vendite, per marca e per anno

vendite_per_marca_anno_df = vendite_df\
                    .join(prodotti_df, vendite_df["Prodotto"] == prodotti_df["ID"])\
                    .groupBy(F.col("Marca"), F.col("Anno"))\
                    .agg(F.count("*"))\
                    .orderBy("Anno", "Marca")

vendite_per_marca_anno_df.show()

+--------+----+--------+
|   Marca|Anno|count(1)|
+--------+----+--------+
|  Fender|2022|       3|
|  Gibson|2022|       5|
|  Ibanez|2022|       6|
|MusicMan|2022|       1|
|  Fender|2023|       7|
|  Gibson|2023|       4|
|  Ibanez|2023|       2|
|MusicMan|2023|       2|
+--------+----+--------+



In [10]:
# Numero di vendite, per venditore

vendite_per_venditore_df = vendite_df\
                    .join(venditori_df, vendite_df["Venditore"] == venditori_df["ID"])\
                    .groupBy(F.col("Venditore"), F.col("Nome"), F.col("Cognome"))\
                    .agg(F.count("*").alias("Numero vendite"))\
                    .select("Nome", "Cognome", "Numero vendite")\
                    .orderBy(F.col("Numero vendite").desc())
                    
vendite_per_venditore_df.show()

+--------+--------+--------------+
|    Nome| Cognome|Numero vendite|
+--------+--------+--------------+
|   Bruno| Bettini|             5|
|   Ivana|  Itteri|             4|
|  Andrea|  Assini|             4|
|   Dario|  Desini|             4|
|   Carlo|Cicerini|             4|
|  Giulia|Giorgini|             3|
|Federica| Fierini|             3|
|   Elena| Ettorre|             3|
+--------+--------+--------------+



In [12]:
# Numero di vendite, per venditore e per anno

vendite_per_venditore_anno_df = vendite_df\
                    .join(venditori_df, vendite_df["Venditore"] == venditori_df["ID"])\
                    .groupBy(F.col("Anno"), F.col("Venditore"), F.col("Nome"), F.col("Cognome"))\
                    .agg(F.count("*").alias("Numero vendite"))\
                    .select("Anno", "Nome", "Cognome", "Numero vendite")\
                    .orderBy(F.col("Anno"), F.col("Numero vendite").desc())

vendite_per_venditore_anno_df.show()

+----+--------+--------+--------------+
|Anno|    Nome| Cognome|Numero vendite|
+----+--------+--------+--------------+
|2022|   Bruno| Bettini|             3|
|2022|   Carlo|Cicerini|             3|
|2022|   Elena| Ettorre|             2|
|2022|   Ivana|  Itteri|             2|
|2022|  Andrea|  Assini|             2|
|2022|  Giulia|Giorgini|             1|
|2022|Federica| Fierini|             1|
|2022|   Dario|  Desini|             1|
|2023|   Dario|  Desini|             3|
|2023|   Ivana|  Itteri|             2|
|2023|Federica| Fierini|             2|
|2023|   Bruno| Bettini|             2|
|2023|  Andrea|  Assini|             2|
|2023|  Giulia|Giorgini|             2|
|2023|   Carlo|Cicerini|             1|
|2023|   Elena| Ettorre|             1|
+----+--------+--------+--------------+



In [13]:
# Ricavo per vendita

ricavo_per_vendita_df = vendite_df\
                    .join(prodotti_df, vendite_df["Prodotto"] == prodotti_df["ID"])\
                    .withColumn("Ricavo vendita", F.col("Quantita")*F.col("Prezzo"))\
                    .select("Data", "Ricavo vendita")

ricavo_per_vendita_df.show()

+----------+--------------+
|      Data|Ricavo vendita|
+----------+--------------+
|2022-01-11|        2300.0|
|2022-02-21|        3000.0|
|2022-03-24|        6000.0|
|2022-04-02|        6800.0|
|2022-04-12|        3800.0|
|2022-05-17|        1800.0|
|2022-05-21|        1800.0|
|2022-06-11|        9200.0|
|2022-07-12|        2000.0|
|2022-08-15|        1800.0|
|2022-09-20|        3800.0|
|2022-10-05|        3600.0|
|2022-11-27|        8500.0|
|2022-11-29|        1000.0|
|2022-12-04|        1800.0|
|2023-01-11|       10000.0|
|2023-02-21|        3600.0|
|2023-03-24|        6000.0|
|2023-04-02|        5700.0|
|2023-04-12|        5100.0|
+----------+--------------+
only showing top 20 rows



In [16]:
# Ricavo totale per venditore

ricavo_totale_per_venditore_df = vendite_df\
                    .join(venditori_df, vendite_df["Venditore"] == venditori_df["ID"])\
                    .join(prodotti_df, vendite_df["Prodotto"] == prodotti_df["ID"])\
                    .withColumn("Ricavo vendita", F.col("Quantita")*F.col("Prezzo"))\
                    .groupBy("Venditore", "Nome", "Cognome")\
                    .agg(F.sum("Ricavo vendita").alias("Ricavo totale"))\
                    .select("Nome", "Cognome", "Ricavo totale")\
                    .orderBy(F.col("Ricavo totale").desc())

ricavo_totale_per_venditore_df.show()

+--------+--------+-------------+
|    Nome| Cognome|Ricavo totale|
+--------+--------+-------------+
|   Elena| Ettorre|      24000.0|
|   Carlo|Cicerini|      20900.0|
|   Dario|  Desini|      20200.0|
|   Ivana|  Itteri|      19900.0|
|   Bruno| Bettini|      16800.0|
|  Andrea|  Assini|      15600.0|
|  Giulia|Giorgini|      11100.0|
|Federica| Fierini|       4300.0|
+--------+--------+-------------+



In [19]:
# Numero di prodotti diversi venduti ogni anno

prodotti_diversi_per_anno_df = vendite_df\
                    .join(venditori_df, vendite_df["Venditore"] == venditori_df["ID"])\
                    .join(prodotti_df, vendite_df["Prodotto"] == prodotti_df["ID"])\
                    .groupBy("Anno")\
                    .agg(F.countDistinct("Marca", "Modello"))\
                    .orderBy("Anno")

prodotti_diversi_per_anno_df.show()

+----+---------------------+
|Anno|count(Marca, Modello)|
+----+---------------------+
|2022|                    9|
|2023|                   11|
+----+---------------------+

